# Description

This notebook shows the various functionality of our internal config

# Imports

In [16]:
import core.config as cconfig

# Initialization

- The default config modes are
    - `update_mode` = "assert_on_overwrite"
        - We can add new keys to the config regardless of the update mode.

    - `clobber_mode` = "assert_on_write_after_use"

    - `report_mode` = "verbose_log_error"

    - `unused_variables_mode` = "warning_on_error",

In [17]:
system_log_dir = "/shared_data/CMTask5198_log_dir_1"
id_col = "asset_id"
universe_version = "v7.1"
vendor = "CCXT"
mode = "trade"
test_asset_id = 1467591036
bar_duration = "5T"
expected_num_child_orders = [0, 5]
use_historical = True
config_dict = {
    "meta": {"id_col": id_col, "use_historical": use_historical},
    "system_log_dir": system_log_dir,
    "ohlcv_market_data": {
        "vendor": vendor,
        "mode": mode,
        "universe": {
            "universe_version": universe_version,
            "test_asset_id": test_asset_id,
        },
    },
}
config = cconfig.Config.from_dict(config_dict)
print(config)

meta: 
  id_col: asset_id
  use_historical: True
system_log_dir: /shared_data/CMTask5198_log_dir_1
ohlcv_market_data: 
  vendor: CCXT
  mode: trade
  universe: 
    universe_version: v7.1
    test_asset_id: 1467591036


In [18]:
# Converting a config to a dict.
dict1 = config.to_dict()
dict1

OrderedDict([('meta',
              OrderedDict([('id_col', 'asset_id'), ('use_historical', True)])),
             ('system_log_dir', '/shared_data/CMTask5198_log_dir_1'),
             ('ohlcv_market_data',
              OrderedDict([('vendor', 'CCXT'),
                           ('mode', 'trade'),
                           ('universe',
                            OrderedDict([('universe_version', 'v7.1'),
                                         ('test_asset_id', 1467591036)]))]))])

In [19]:
# Create a config from a flattened dict.
config = cconfig.Config._get_config_from_flattened_dict(dict1)
print(config)

meta: 
  id_col: asset_id
  use_historical: True
system_log_dir: /shared_data/CMTask5198_log_dir_1
ohlcv_market_data: 
  vendor: CCXT
  mode: trade
  universe: 
    universe_version: v7.1
    test_asset_id: 1467591036


# Get and Set values

In [20]:
# Chain Accessing.
print("id_col - ", config["meta", "id_col"])
# Nested Accessing.
print("vendor - ", config["ohlcv_market_data"]["vendor"])

id_col -  asset_id
vendor -  CCXT


In [21]:
# Access a exsisting key to check a default value is not returned.
config.get("system_log_dir", "default_value")

'/shared_data/CMTask5198_log_dir_1'

In [22]:
# Access a exsisting key to check a default value is not returned
# but the type is checked. Should raise an exception because
# `system_log_dir` is of the type `str`.
config.get("system_log_dir", "default_value", int)

AssertionError: 
################################################################################
* Failed assertion *
Instance of '/shared_data/CMTask5198_log_dir_1' is '<class 'str'>' instead of '<class 'int'>'
################################################################################


In [23]:
#  Set a key that doesn't exist.
config["bar_duration"] = "5T"
print(config)

meta: 
  id_col: asset_id
  use_historical: True
system_log_dir: /shared_data/CMTask5198_log_dir_1
ohlcv_market_data: 
  vendor: CCXT
  mode: trade
  universe: 
    universe_version: v7.1
    test_asset_id: 1467591036
bar_duration: 5T


In [24]:
# Set a key that already exists.
# Need to update the mode of config to `overwrite`
config.update_mode = "overwrite"
config["bar_duration"] = "6T"
print(config)

meta: 
  id_col: asset_id
  use_historical: True
system_log_dir: /shared_data/CMTask5198_log_dir_1
ohlcv_market_data: 
  vendor: CCXT
  mode: trade
  universe: 
    universe_version: v7.1
    test_asset_id: 1467591036
bar_duration: 6T


# Update

In [56]:
# Test the update functionality.
# Update with no common vlaues in configs.
config1 = cconfig.Config()
config_tmp = config1.add_subconfig("read_data")
config_tmp["file_name"] = "foo_bar.txt"
config_tmp["nrows"] = 999
config1["single_val"] = "hello"
config_tmp = config1.add_subconfig("zscore")
config_tmp["style"] = "gaz"
config_tmp["com"] = 28
print(config1)
print("*" * 60)
#
config2 = cconfig.Config()
config_tmp = config2.add_subconfig("write_data")
config_tmp["file_name"] = "baz.txt"
config_tmp["nrows"] = 999
config2["single_val2"] = "goodbye"
print(config2)

read_data: 
  file_name: foo_bar.txt
  nrows: 999
single_val: hello
zscore: 
  style: gaz
  com: 28
************************************************************
write_data: 
  file_name: baz.txt
  nrows: 999
single_val2: goodbye


In [61]:
# With default `update_mode=assert_on_overwirte` it
# will raise an assertion.
config1.update(config2, update_mode="overwrite")
print(config1)

read_data: 
  file_name: foo_bar.txt
  nrows: 999
single_val: hello
zscore: 
  style: gaz
  com: 28
write_data: 
  file_name: baz.txt
  nrows: 999
single_val2: goodbye


In [59]:
# Update with common values in configs.
config3 = cconfig.Config()
config_tmp = config3.add_subconfig("read_data")
config_tmp["file_name"] = "foo_bar.txt"
config_tmp["nrows"] = 999
config3["single_val"] = "hello"
config_tmp = config3.add_subconfig("zscore")
config_tmp["style"] = "gaz"
config_tmp["com"] = 28
print(config3)
print("*" * 60)
#
config4 = cconfig.Config()
config_tmp = config4.add_subconfig("read_data")
config_tmp["file_name"] = "baz.txt"
config_tmp["nrows"] = 999
config4["single_val"] = "goodbye"
config_tmp = config4.add_subconfig("zscore")
config_tmp["style"] = "super"
config_tmp = config4.add_subconfig("extra_zscore")
config_tmp["style"] = "universal"
config_tmp["tau"] = 32
print(config4)

read_data: 
  file_name: foo_bar.txt
  nrows: 999
single_val: hello
zscore: 
  style: gaz
  com: 28
************************************************************
read_data: 
  file_name: baz.txt
  nrows: 999
single_val: goodbye
zscore: 
  style: super
extra_zscore: 
  style: universal
  tau: 32


In [62]:
config3.update(config4, update_mode="overwrite")
print(config3)

read_data: 
  file_name: baz.txt
  nrows: 999
single_val: goodbye
zscore: 
  style: super
  com: 28
extra_zscore: 
  style: universal
  tau: 32


In [63]:
# Behaviour when update mode is `assign_if_missing`.
# Existing keys will not be modified and no exception
# is raised. New keys are added if not present.
config5 = cconfig.Config()
config5["read_data", "file_name"] = "hello"
config5["read_data2"] = "world"
config3.update(config5, update_mode="assign_if_missing")
print(config3)

Value 'baz.txt' for key 'file_name' already exists. Not overwriting with 'hello' since update_mode=assign_if_missing


read_data: 
  file_name: baz.txt
  nrows: 999
single_val: goodbye
zscore: 
  style: super
  com: 28
extra_zscore: 
  style: universal
  tau: 32
read_data2: world


In [28]:
# To update a read only config, set `value=False`.
config.update_mode = "overwrite"
config.clobber_mode = "allow_write_after_use"
config.mark_read_only(value=False)
config["bar_duration"] = "7T"
print(config)

meta: 
  id_col: asset_id
  use_historical: True
system_log_dir: /shared_data/CMTask5198_log_dir_1
ohlcv_market_data: 
  vendor: CCXT
  mode: trade
  universe: 
    universe_version: v7.1
    test_asset_id: 1467591036
bar_duration: 7T


# Read Only Config

In [29]:
# Test config read-only property.
# Cannot set or update a value that doesn't exists on a read-only config.
# Should raise an exception.
config.update_mode = "overwrite"
config.clobber_mode = "allow_write_after_use"
config.mark_read_only()
config["bar_duration"] = "7T"

exception=Can't set key='bar_duration' to val='7T' in read-only config
self=
  meta: 
    id_col: asset_id
    use_historical: True
  system_log_dir: /shared_data/CMTask5198_log_dir_1
  ohlcv_market_data: 
    vendor: CCXT
    mode: trade
    universe: 
      universe_version: v7.1
      test_asset_id: 1467591036
  bar_duration: 7T
key='bar_duration'
config=
  meta: 
    id_col: asset_id
    use_historical: True
  system_log_dir: /shared_data/CMTask5198_log_dir_1
  ohlcv_market_data: 
    vendor: CCXT
    mode: trade
    universe: 
      universe_version: v7.1
      test_asset_id: 1467591036
  bar_duration: 7T


ReadOnlyConfigError: Can't set key='bar_duration' to val='7T' in read-only config
self=
  meta: 
    id_col: asset_id
    use_historical: True
  system_log_dir: /shared_data/CMTask5198_log_dir_1
  ohlcv_market_data: 
    vendor: CCXT
    mode: trade
    universe: 
      universe_version: v7.1
      test_asset_id: 1467591036
  bar_duration: 7T

# Get and Mark as used

In [30]:
# Test verbose mode with `marked_as_used` == True.
# The leaf keys in heirarchy will be marked as `True`
mode = "verbose"
config.get_and_mark_as_used("meta")
config.to_string(mode)

'meta (marked_as_used=False, writer=None, val_type=core.config.config_.Config): \n  id_col (marked_as_used=True, writer=/tmp/ipykernel_411/3453495586.py::4::<module>, val_type=str): asset_id\n  use_historical (marked_as_used=True, writer=/tmp/ipykernel_411/3453495586.py::4::<module>, val_type=bool): True\nsystem_log_dir (marked_as_used=False, writer=None, val_type=str): /shared_data/CMTask5198_log_dir_1\nohlcv_market_data (marked_as_used=False, writer=None, val_type=core.config.config_.Config): \n  vendor (marked_as_used=False, writer=None, val_type=str): CCXT\n  mode (marked_as_used=False, writer=None, val_type=str): trade\n  universe (marked_as_used=False, writer=None, val_type=core.config.config_.Config): \n    universe_version (marked_as_used=False, writer=None, val_type=str): v7.1\n    test_asset_id (marked_as_used=False, writer=None, val_type=int): 1467591036\nbar_duration (marked_as_used=False, writer=None, val_type=str): 7T'

In [31]:
# Get and mark a value as used.
config.get_and_mark_as_used("bar_duration")

'7T'

In [32]:
# Check if the value is marked as used.
config.get_marked_as_used("bar_duration")

True

# General Functionality

In [33]:
# Convert a config to a python code.
code = config.to_python()
print(code)
print("\n")
print(type(code))

Config([('meta', Config([('id_col', 'asset_id'), ('use_historical', True)])), ('system_log_dir', '/shared_data/CMTask5198_log_dir_1'), ('ohlcv_market_data', Config([('vendor', 'CCXT'), ('mode', 'trade'), ('universe', Config([('universe_version', 'v7.1'), ('test_asset_id', 1467591036)]))])), ('bar_duration', '7T')])


<class 'str'>


In [34]:
# Build a config from a python code.
config2 = cconfig.Config.from_python(code)
print(config2)
print("\n")
print(type(config2))

meta: 
  id_col: asset_id
  use_historical: True
system_log_dir: /shared_data/CMTask5198_log_dir_1
ohlcv_market_data: 
  vendor: CCXT
  mode: trade
  universe: 
    universe_version: v7.1
    test_asset_id: 1467591036
bar_duration: 7T


<class 'core.config.config_.Config'>


In [35]:
# Testing the flatten functionality.
config.flatten()

OrderedDict([(('meta', 'id_col'), 'asset_id'),
             (('meta', 'use_historical'), True),
             (('system_log_dir',), '/shared_data/CMTask5198_log_dir_1'),
             (('ohlcv_market_data', 'vendor'), 'CCXT'),
             (('ohlcv_market_data', 'mode'), 'trade'),
             (('ohlcv_market_data', 'universe', 'universe_version'), 'v7.1'),
             (('ohlcv_market_data', 'universe', 'test_asset_id'), 1467591036),
             (('bar_duration',), '7T')])

In [36]:
# Test a config that can be serialized correctly.
config.is_serializable()

True

In [37]:
# Keep or skip empty leaves.
config.to_dict(keep_leaves=False)

OrderedDict([('meta',
              OrderedDict([('id_col', 'asset_id'), ('use_historical', True)])),
             ('system_log_dir', '/shared_data/CMTask5198_log_dir_1'),
             ('ohlcv_market_data',
              OrderedDict([('vendor', 'CCXT'),
                           ('mode', 'trade'),
                           ('universe',
                            OrderedDict([('universe_version', 'v7.1'),
                                         ('test_asset_id', 1467591036)]))])),
             ('bar_duration', '7T')])

In [38]:
# Convert a config to a string.
mode = "verbose"
print(config.to_string(mode="verbose"))

meta (marked_as_used=False, writer=None, val_type=core.config.config_.Config): 
  id_col (marked_as_used=True, writer=/tmp/ipykernel_411/3453495586.py::4::<module>, val_type=str): asset_id
  use_historical (marked_as_used=True, writer=/tmp/ipykernel_411/3453495586.py::4::<module>, val_type=bool): True
system_log_dir (marked_as_used=False, writer=None, val_type=str): /shared_data/CMTask5198_log_dir_1
ohlcv_market_data (marked_as_used=False, writer=None, val_type=core.config.config_.Config): 
  vendor (marked_as_used=False, writer=None, val_type=str): CCXT
  mode (marked_as_used=False, writer=None, val_type=str): trade
  universe (marked_as_used=False, writer=None, val_type=core.config.config_.Config): 
    universe_version (marked_as_used=False, writer=None, val_type=str): v7.1
    test_asset_id (marked_as_used=False, writer=None, val_type=int): 1467591036
bar_duration (marked_as_used=True, writer=/tmp/ipykernel_411/3598366333.py::2::<module>, val_type=str): 7T
